## FGSW网络对抗攻击
<hr>

FGSW(Fast Gradient sign Methods)梯度符号攻击是对抗性攻击领域，尤其是白盒攻击领域的鼻祖之作，现在我们用Mindspore来实现FGSM

### 对抗样本定义

Szegedy在2013年最早提出对抗样本的概念:

$$\hat{x}=x+\epsilon sign(\nabla_{x}J(\theta,x,y))$$

<img src="img\FGSM1.jpg">

<hr>

### 训练模型

本案例将使用MNIST训练一个进度达标的LeNet网络，然后运行上文中所提到的FGSM攻击方法，达到欺骗网络模型，让模型实现错误分类的效果，由于在另一篇文章里已经详细介绍了LeNet网络的训练，这里就快速过一遍

In [49]:
from mindvision.dataset import Mnist
import mindspore as ms
import mindspore.nn as nn
from mindvision.engine.callback import LossMonitor
from sklearn import config_context

class LeNet5(nn.Cell):
    def __init__(self,num_class=10,num_channel=1):
        super(LeNet5,self).__init__()
        # 卷积层1，通道：1-6 卷积核大小：5*5 
        self.conv1=nn.Conv2d(num_channel,6,5,pad_mode='valid')
        # 卷积层2，通道：6-16 卷积核大小：5*5 
        self.conv2=nn.Conv2d(6,16,5,pad_mode='valid')
        # 三个全连接层Dense
        self.fc1=nn.Dense(16*5*5,120)
        self.fc2=nn.Dense(120,84)
        self.fc3=nn.Dense(84,num_class)
        self.relu=nn.ReLU()
        self.max_pool2d=nn.MaxPool2d(kernel_size=2,stride=2)
        self.flatten=nn.Flatten()
    
    def construct(self,x):
        # 构建前向网络
        x=self.conv1(x) # C:1-6 N:32-5+1=28
        x=self.relu(x)
        x=self.max_pool2d(x) # C:6-6 N:28/2=14
        x=self.conv2(x) # C:6-16 N: 14-5+1=10
        x = self.relu(x)
        x = self.max_pool2d(x) # C:16-16 n:10/2=5
        x = self.flatten(x) # 16,5,5->16*5*5
        x = self.fc1(x) # 16*5*5->120
        x = self.relu(x)
        x = self.fc2(x) # 120->84
        x = self.relu(x)
        x = self.fc3(x) # 84->10
        return x
# 创建网络对象
network=LeNet5(num_class=10)

# 加载数据集
download_train=Mnist(path="./mnist",split="train",shuffle=True,download=True)
download_eval=Mnist(path="./mnist",split="test",download=True)
dataset_train=download_train.run()
dataset_eval=download_eval.run()

# 定义优化器和损失函数
net_loss=nn.SoftmaxCrossEntropyWithLogits(sparse=True,reduction='mean')
net_opt=nn.Momentum(network.trainable_params(),learning_rate=0.01,momentum=0.9)

# 若想查询每一个epoch完毕之后的参数，则启用下面两句来下载check_point
# config_ck=ms.CheckpointConfig(save_checkpoint_steps=1875,keep_checkpoint_max=10)
# ckpoint=ms.ModelCheckpoint(prefix="checkpoint_lent",config=config_ck)

model=ms.Model(network,loss_fn=net_loss,optimizer=net_opt,metrics={'accuracy'})
model.train(5,dataset_train,callbacks=[LossMonitor(0.01,1875)])


[WARNING] ME(9828:16456,MainProcess):2022-08-06-01:37:41.684.242 [mindspore\dataset\engine\datasets_user_defined.py:656] Python multiprocessing is not supported on Windows platform.
[WARNING] ME(9828:16456,MainProcess):2022-08-06-01:37:42.708.10 [mindspore\dataset\engine\datasets_user_defined.py:656] Python multiprocessing is not supported on Windows platform.
[WARNING] ME(9828:16456,MainProcess):2022-08-06-01:37:42.713.10 [mindspore\dataset\core\validator_helpers.py:799] 'Resize' from mindspore.dataset.vision.c_transforms is deprecated from version 1.8 and will be removed in a future version. Use 'Resize' from mindspore.dataset.vision instead.
[WARNING] ME(9828:16456,MainProcess):2022-08-06-01:37:42.718.10 [mindspore\dataset\core\validator_helpers.py:799] 'Rescale' from mindspore.dataset.vision.c_transforms is deprecated from version 1.8 and will be removed in a future version. Use 'Rescale' from mindspore.dataset.vision instead.
[WARNING] ME(9828:16456,MainProcess):2022-08-06-01:37:4

Epoch:[  0/  5], step:[ 1875/ 1875], loss:[2.286/2.302], time:3.001 ms, lr:0.01000
Epoch time: 8666.509 ms, per step time: 4.622 ms, avg loss: 2.302
Epoch:[  1/  5], step:[ 1875/ 1875], loss:[1.439/2.263], time:4.501 ms, lr:0.01000
Epoch time: 10671.359 ms, per step time: 5.691 ms, avg loss: 2.263
Epoch:[  2/  5], step:[ 1875/ 1875], loss:[0.004/0.166], time:30.005 ms, lr:0.01000
Epoch time: 10832.887 ms, per step time: 5.778 ms, avg loss: 0.166
Epoch:[  3/  5], step:[ 1875/ 1875], loss:[0.012/0.063], time:3.000 ms, lr:0.01000
Epoch time: 11442.493 ms, per step time: 6.103 ms, avg loss: 0.063
Epoch:[  4/  5], step:[ 1875/ 1875], loss:[0.112/0.049], time:2.501 ms, lr:0.01000
Epoch time: 11266.963 ms, per step time: 6.009 ms, avg loss: 0.049


> 网络训练完毕，让我们看看训练完网络的准确性


In [43]:
acc=model.eval(dataset_eval)
print("{}".format(acc))

{'accuracy': 0.9730568910256411}


<hr>

### FGSM攻击

在得到精准的LeNet网络之后，下面将会采用FGSM攻击方法，在图像中加载噪声之后重新测试，先通过损失函数求反向梯度：



In [44]:
import mindspore.ops as ops

class WithLossCell(nn.Cell):
    # 包装网络损失函数
    def __init__(self, network,loss_fn):
        super(WithLossCell,self).__init__()
        self._network=network
        self._loss_fn=loss_fn

    def construct(self,data,label):
        out=self._network(data)
        return self._loss_fn(out,label)

class GradWrapWithLoss(nn.Cell):
    def __init__(self,network):
        super(GradWrapWithLoss,self).__init__()
        self._grad_all=ops.composite.GradOperation(get_all=True,sens_param=False)
        self._network=network
    
    def construct(self,inputs,labels):
        gout=self._grad_all(self._network)(inputs,labels)
        return gout[0]

> 构造完损失函数和求梯度类之后即可利用上面的公式进行攻击

$$\hat{x}=x+\epsilon sign(\nabla_{x}J(\theta,x,y))$$

In [45]:
import numpy as np

class FastGradientSignMethod:
    def __init__(self,network,eps=0.07,loss_fn=None):
        self._network=network
        self._eps=eps
        with_loss_cell=WithLossCell(self._network,loss_fn)
        self._grad_all=GradWrapWithLoss(with_loss_cell)
        self._grad_all.set_train()
    
    def _gradient(self,inputs,labels):
        # 求出梯度
        out_grad=self._grad_all(inputs,labels)
        gradient=out_grad.asnumpy()
        gradient=np.sign(gradient)
        return gradient
    
    def generate(self,inputs,labels):
        # 实现FGSM
        inputs_tensor=ms.Tensor(inputs)
        labels_tensor=ms.Tensor(labels)
        gradient=self._gradient(inputs_tensor,labels_tensor)
        # 产生扰动
        perturbation=self._eps*gradient
        adv_x=inputs+perturbation
        return adv_x

    def batch_generate(self,inputs,labels,batch_size=32):
        # 对数据集进行处理
        arr_x=inputs
        arr_y=labels
        len_x=len(inputs)
        batches=int(len_x/batch_size) # 计算batch数
        res=[]
        for i in range(batches):
            x_batch=arr_x[i*batch_size:(i+1)*batch_size]
            y_batch=arr_y[i*batch_size:(i+1)*batch_size]
            adv_x=self.generate(x_batch,y_batch)
            res.append(adv_x)
        adv_x=np.concatenate(res,axis=0)
        return adv_x

再次处理MINIST数据集中测试集的数据

In [52]:
import numpy as np
import matplotlib.pyplot as plt

images=[]
labels=[]
test_images=[]
test_labels=[]
predict_labels=[]

ds_test=dataset_eval.create_dict_iterator(output_numpy=True)



for data in ds_test:
    images=data['image'].astype(np.float32)
    labels=data['label']
    test_images.append(images)
    test_labels.append(labels)
    pred_labels=np.argmax(model.predict(ms.Tensor(images)).asnumpy(),axis=1)
    predict_labels.append(pred_labels)

test_images = np.concatenate(test_images)
predict_labels = np.concatenate(predict_labels)
true_labels = np.concatenate(test_labels) # concatenate拉成一个维度
print(true_labels)

[2 1 9 ... 9 0 1]


### 运行攻击

由FGSM攻击公式中可以看出，攻击参数$\epsilon$越大，对梯度的改变就越大。当$\epsilon$为零时则攻击效果不体现。$$\eta=\epsilon sign(\nabla_{x}J(\theta))$$

> $\epsilon$为0时

In [53]:
import mindspore.ops as ops

fgsm = FastGradientSignMethod(network, eps=0.0, loss_fn=net_loss)
advs = fgsm.batch_generate(test_images, true_labels, batch_size=32)

adv_predicts = model.predict(ms.Tensor(advs)).asnumpy()
adv_predicts = np.argmax(adv_predicts, axis=1)
accuracy = np.mean(np.equal(adv_predicts, true_labels))
print(accuracy)

0.9885817307692307


> $\epsilon$为0.9时

In [60]:
fgsm = FastGradientSignMethod(network, eps=0.5, loss_fn=net_loss)
advs = fgsm.batch_generate(test_images, true_labels, batch_size=32)

adv_predicts = model.predict(ms.Tensor(advs)).asnumpy()
adv_predicts = np.argmax(adv_predicts, axis=1)
accuracy = np.mean(np.equal(adv_predicts, true_labels))
print(accuracy)

0.29837740384615385


可以看到，acc变的很低，现在我们来看看受攻击图片的实际形态：

In [1]:

import matplotlib.pyplot as plt

adv_examples = np.transpose(advs[:10], [0, 2, 3, 1])
ori_examples = np.transpose(test_images[:10], [0, 2, 3, 1])

plt.figure(figsize=(10, 3), dpi=120)
for i in range(10):
    plt.subplot(3, 10, i+1)
    plt.axis("off")
    plt.imshow(np.squeeze(ori_examples[i]))
    plt.subplot(3, 10, i+11)
    plt.axis("off")
    plt.imshow(np.squeeze(adv_examples[i]))
plt.show()


NameError: name 'np' is not defined

实际上图片发生的变化不大，人眼还是可以非常清晰明了的识别的，但是在精度测试中缺严重下降，这揭示了神经网络的脆弱性